## Prepare environment

### Install project dependencies

Install the remaining Python dependencies.

In [ ]:
import subprocess

project_dependencies = "dotenv weave langchain_core langchain_openai langchain_ollama langchain-google-genai langchain-groq"

# Try to install using poetry first and then pip
try:
    response = subprocess.check_output(["poetry", "--version"])
    print("Poetry is installed")
    !poetry add $project_dependencies
except:
    print("Poetry is not installed. Using pip to install dependencies")
    %pip install -qU $project_dependencies

### Load API Keys from environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key_preview = os.getenv("OPENAI_API_KEY")[:10]
print(f"First 10 characters of API key: {api_key_preview}")

wandb_key_preview = os.getenv("WANDB_API_KEY")[:10]
print(f"First 10 characters of W&B key: {wandb_key_preview}")

## Run project

### Initialize tracking with Weave

In [ ]:
import weave
weave.init("synthetic_ticket_dataset_generation")

### Choose a model

In [ ]:
from langchain_groq import ChatGroq

# Initialize the chat model
chat_model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0.6,
)

In [ ]:
def parse_response(prompt, response):
    content = response.content
    ## Remove everything between <think> and </think>
    think_start = content.find("<think>")
    think_end = content.find("</think>")
    thought = content[think_start + len("<think>"):think_end]
    final_answer = content[think_end + len("</think>"):]
    parsed_response = {
        "prompt" : str(prompt).strip(),
        "final_answer": final_answer.strip(),
        "thought" : thought.strip()
    }
    return parsed_response

def parse_responses(prompts, responses):
    parsed_responses = []
    for prompt, response in zip(prompts, responses):
        parsed_response = parse_response(prompt, response)
        parsed_responses.append(parsed_response)
    return parsed_responses

In [ ]:
tipos_chamados = {
    '1' : "Equipamentos de Informática: Solicitações relacionadas a hardware, como computadores, impressoras e periféricos.",
    '2' : "Acesso a Sistemas: Solicitações referentes a problemas de login, senha, permissões ou acesso a plataformas.",
    '3' : "Comunicação e Redes: Solicitações envolvendo infraestrutura de rede, internet, telefonia e outros serviços de comunicação.",
    '4' : "Outros: Problemas que não se enquadram nas categorias acima."
}

tipos_chamados_for_prompt = " \n".join([f"{k}: {v}" for k, v in tipos_chamados.items()])
print(tipos_chamados_for_prompt)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Define the prompt template
prompt_for_classification = ChatPromptTemplate([
    ("system",
     f"""Você é um assistente de IA especializado em classificação de chamados de suporte técnico.
    Classifique o chamado em um dos tipos seguintes conforme a descrição do que engloba cada chamado.
    A resposta deve ser apenas o número do tipo do chamado.
    {tipos_chamados_for_prompt}"""
    ),
    ("user", "{chamado}")
])

In [ ]:
chamados = [
    "O computador não liga.",
    # "Não consigo acessar o sistema.",
    # "A internet está lenta.",
    # "O telefone não funciona."
]

responses = []
for chamado in chamados:
    response = chat_model.invoke(prompt_for_classification.invoke({"chamado": chamado}))
    responses.append(response)

In [ ]:
import json

with open("classification.json", "w") as f:
    answers = parse_responses(chamados, responses)
    json.dump(answers, f, indent=2)

In [ ]:
variedade_problemas = [
    "Ambiente Corporativo: Problema ocorrido em um ambiente de escritório com múltiplos usuários.",
    "Home Office: Questões relatadas por usuários que trabalham remotamente.",
    "Infraestrutura Crítica: Problemas que afetam sistemas essenciais ou serviços críticos.",
    "Ocorrência Intermitente: Problemas que surgem de forma esporádica e sem padrão definido.",
    "Impacto Regional: Problemas que afetam não apenas um usuário, mas uma área ou setor específico.",
    "Complexidade Elevada: Problemas com múltiplas causas ou que exigem uma solução abrangente."
]

componentes_afetados = {
    "1": [
        "Hardware",
        "Sistema Operacional",
        "Outros"
    ],
    "2": [
        "Aplicativo Corporativo",
        "Segurança da Informação",
        "Base de Dados",
        "Outros"
    ],
    "3": [
        "Rede e Conectividade",
        "Segurança da Informação",
        "Outros"
    ],
    "4": [
        "Outros"
    ]
}

In [ ]:
prompt_for_generation = ChatPromptTemplate([
    ("system",
     """Você é um assistente de IA especialista em suporte técnico e na classificação de chamados. 
Seu objetivo é simular um exemplo realista de chamado de suporte técnico, utilizando as informações fornecidas.

Tipos de Chamados Disponíveis:
{tipos_chamados_for_prompt}

Instruções:
1. O chamado deve corresponder ao tipo indicado pelo usuário.
2. A situação deve envolver o componente mencionado pelo usuário.
3. O chamado deve conter a especifidade do problema
3. Gere um exemplo coerente e realista contendo apenas um título e uma descrição.
4. Formate a resposta exatamente como:
   "<Título>"; "<Descrição>"
5. Não adicione informações extras, comentários ou conteúdo além do solicitado.

Exemplo:
"Erro ao acessar o sistema de RH"; "Ao tentar acessar o sistema de RH, recebi uma mensagem de erro que indica uma falha no aplicativo corporativo."

Produza um único exemplo de chamado.
"""
    ),
    ("user", "Tipo: {tipo}\nComponente Afetado: {componente_afetado}\nEspecifidade do Problema:{detalhe_problema}")
])


In [ ]:
import random, time

number_of_tickets_required = 5000

try:
    with open("generation.json", "r") as f:
      ticket_analysis_results = json.loads(f.read())
      progress = len(ticket_analysis_results) / number_of_tickets_required * 100
      print(f"Resuming from saved progress: {progress:.2f}%")
except:
    ticket_analysis_results = []

number_of_tickets_generated = len(ticket_analysis_results)

tipos_solicitados = random.choices(list(tipos_chamados.keys()), k= number_of_tickets_required-number_of_tickets_generated)
number_tries_rate_limit = 0

for i in range(len(tipos_solicitados)):
    tipo = tipos_solicitados[i]
    try:
        response = chat_model.invoke(
            prompt_for_generation.invoke({
              "tipos_chamados_for_prompt": tipos_chamados_for_prompt,
              "tipo": tipo,
              "componente_afetado": random.choice(list(componentes_afetados[f'{tipo}'])),
              "detalhe_problema": random.choice(variedade_problemas)}))
    except Exception as e:
        error_message = str(e)
        # print(f"Error: {error_message}")
        if "429" in error_message:
            number_tries_rate_limit += 1
            print('\r\x1b[2K',
                  f"\rProgress: {progress:.2f}%",
                  f" - Rate limit exceeded. Waiting for {number_tries_rate_limit} minutes.", end="")
            time.sleep(60)
            i -= 1
    else:
        try:
            with open("generation.json", "r") as f:
                saved_answers = json.loads(f.read())
        except:
            saved_answers = []
        with open("generation.json", "w") as f:
            answers = parse_response(tipo, response)
            saved_answers.append(answers)
            json.dump(saved_answers, f, indent=2)
        time.sleep(3)
        number_tries_rate_limit = 0
        progress = len(saved_answers) / number_of_tickets_required * 100
        print('\r\x1b[2K', f"\rProgress: {progress:.2f}%", end="")
    

## Check generated dataset

### Installing Ollama dependencies

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`

In [ ]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules
# if IN_COLAB:
#   !sudo apt update -qq
#   !sudo apt install -qq -y pciutils
#   !curl -fsSL https://ollama.com/install.sh | sh
# else:
#     print("Not running in Google Colab")
#     ! if ! ollama --version; then echo "ollama is not installed" && exit 1; fi

### Starting Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Because Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [ ]:
# import threading
# import subprocess
# import time
# import requests

# def run_ollama_serve():
#   subprocess.Popen(["ollama", "serve"])

# # Check if ollama is running
# try:
#   response = requests.get('http://localhost:11434')
#   if response.status_code == 200:
#     print("Ollama is running")
# except:
#   print("Ollama is not running")
#   thread = threading.Thread(target=run_ollama_serve)
#   thread.start()
#   time.sleep(5)

### Choose model

In [ ]:
## Uncomment the following code to change to Ollama

# from langchain_ollama import ChatOllama

# # Initialize the chat model
# model = 'deepseek-r1:14b'
# !ollama pull $model
# chat_model = ChatOllama(
#     model=model,  # Specify the model version
#     base_url="http://localhost:11434",  # URL where Ollama is running locally
#     temperature=0.6,  # Control the randomness of the output (0.0 to 1.0)
# )

In [ ]:
import json
with open("generation.json", "r") as f:
    answers = json.loads(f.read())
    print(f"Total tickets generated: {len(answers)}")
    tickets_sample = [{'final_answer': ticket['final_answer'], 'prompt' : ticket['prompt']} for ticket in answers]

print("Analyzing generated tickets...")
try:
    with open("generation_analysed.json", "r") as f:
        ticket_analysis_results = json.loads(f.read())
except:
    ticket_analysis_results = []
    
progress = len(ticket_analysis_results) / len(tickets_sample) * 100
print(f"Resuming from saved progress: {progress:.2f}%")
print("Total tickets analyzed:", len(ticket_analysis_results))
print("Total tickets to analyze:", len(tickets_sample))

number_tries_rate_limit = 0

for i in range(len(ticket_analysis_results), len(tickets_sample)):
    tickets = tickets_sample[i]
    ticket_text = tickets['final_answer']
    ticket_class = tickets['prompt']
    try:
        response = chat_model.invoke(prompt_for_classification.invoke({"chamado": ticket_text}))
    except Exception as e:
        error_message = str(e)
        # print(f"Error: {error_message}")
        if "429" in error_message:
            number_tries_rate_limit += 1
            print('\r\x1b[2K',
                f"\rProgress: {progress:.2f}%",
                f" - Rate limit exceeded. Waiting for {number_tries_rate_limit} minutes.", end="")
            time.sleep(60)
            i -= 1
    else:
        with open("generation_analysed.json", "w") as f:
            answers = parse_response(ticket_text, response)
            answers['correct'] = (answers['final_answer'] == ticket_class)
            ticket_analysis = {
                "ticket": ticket_text,
                "correct": answers['correct'],
                "prompted": answers['final_answer'],
                "analysed": ticket_class
                
            }
            ticket_analysis_results.append(ticket_analysis)
            json.dump(ticket_analysis_results, f, indent=2)
        number_tries_rate_limit = 0
        progress = len(ticket_analysis_results) / len(tickets_sample) * 100
        print('\r\x1b[2K', f"\rProgress: {progress:.2f}%", end="")


In [ ]:
import json

divergent_classification = []
with open("check_generated_tickets.json", "w") as f:
    for i in range(len(responses)):
        answer = parse_response(tickets_sample[i]['prompt'], responses[i])
        if answer['final_answer'] != answer['prompt']:
            divergent_classification.append(
                {
                    "prompt": answer['prompt'],
                    "analysis": answer['final_answer'],
                    "ticket_text": tickets_sample[i]['final_answer']
                }
            ) 
    json.dump(divergent_classification, f, indent=2)

### Other

In [ ]:
# from langchain_ollama.chat_models import ChatOllama

# # Initialize the ChatOllama model
# model_llama = ChatOllama(
#     model=model,  # Specify the model version
#     base_url="http://localhost:11434",  # URL where Ollama is running locally
#     temperature=0.7,  # Control the randomness of the output (0.0 to 1.0)
# )

# # Note: Ensure Ollama is running on your computer before executing this code

# # If you encounter an OllamaEndpointNotFoundError, you may need to pull the model
# # Run the following command in your terminal:
# # ollama pull llama3.1

# # Generate a response from the model
# response = model_llama.invoke("Olá, meu nome é Yuri. Qual é o seu nome?")

# # Print the response
# print(response)